<a href="https://colab.research.google.com/github/gps-prasad/AgenticAI/blob/main/chatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%pip install langgraph

In [116]:
from google.colab import userdata
import torch
import gradio as gr
from typing import TypedDict, Optional
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from langgraph.graph import StateGraph, START, END

In [25]:
hf_token = userdata.get("HF_TOKEN")
login(hf_token)

In [32]:
model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [113]:
system_prompt  = "You are a friendly chatbot who always responds in a funny way"

In [114]:
def message(message,history=[]):
  messages = [{"role": "system", "content": system_prompt}] + history + [{"role":"user", "content":message}]

  inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
  ).to(model.device)

  outputs = model.generate(**inputs, max_new_tokens=400)

  try:
    index = len(outputs[0]) - outputs[0].tolist()[::-1].index(151668)
  except ValueError:
    index = 0

  content = tokenizer.decode(outputs[0][index:], skip_special_tokens=True).strip("\n")

  return content

In [119]:
demo = gr.ChatInterface(fn=message)
demo.launch()

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e393be2040fdb973f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
